# InfoGAN
<center><img src="https://drive.google.com/uc?id=1_eHGgFAACgcf80S1Oi_4Qax2nPxQP6O2" width="700" height="350"></center>

Latent 코드 $c$ 는 생성기(Generator) 입력의 일부분이며 각 분포를 나타내는 변수(continuous, categorial)를 포함하고 있다.

제너레이터가 Latent 코드 c를 사용하도록 하기 위해, 상호정보량개념(mutual information)이 GAN의 손실함수에 도입된다. 

mutual information(I)는 Y가 주어진 X의 양을 측정하거나 그 반대도 측정한다. 이것은 아래와 같이 정의된다.  

---


$$I(X;Y) = entropy(X) - entropy(X|Y) = entropy(Y) - entropy(Y|X) $$



---

The InfoGAN loss is: 


---

$$\min_{G} \max_{D} \, V(D, G) - \lambda I(c, G(z, c))$$


---

여기서 $V(D,G)$ 는 GAN의 손실함수이다. $I(c, G(z, c))$는 상호정보량(I)이다(goes in as regularization). 목표는 데이터에 대한 의미있는 코드 c를 배우기 위해 높은 상호정보량에 도달하는 것이다. 

---

Loss functions

*   Categorical code c : SoftmaxCrossEntropyLoss
*   Continious code c : L2Loss
*   Normal GAN loss : SigmoidBinaryCrossEntropyLoss


## TensorFlow 및 기타 라이브러리 가져오기

In [1]:
import tensorflow as tf
slim = tf.contrib.slim
tfd = tf.distributions

from tensorflow.examples.tutorials.mnist import input_data

W0825 16:19:24.515990 140166383638400 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Load the MNIST dataset.

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

W0825 16:19:24.537600 140166383638400 deprecation.py:323] From <ipython-input-2-a839aeb82f4b>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0825 16:19:24.538805 140166383638400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0825 16:19:24.540583 140166383638400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0825 16:19:24.824619 140166383638400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0825 16:19:24.827623 140166383638400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0825 16:19:24.883741 140166383638400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## InfoGAN 네트워크 생성
<center><img src="https://drive.google.com/uc?id=18msRuNcK_oXw2UReSrya0fcpRT-ZKdzZ" width="600" height="200"></center>

논문에 네트워크 구조가 정의되어 있음

In [0]:
def generator(z, out_activation_fn=tf.nn.sigmoid):
  """Generate images from latents."""
  with slim.arg_scope([slim.fully_connected, slim.conv2d_transpose],
                      normalizer_fn=slim.batch_norm):
    with slim.arg_scope([slim.conv2d_transpose], kernel_size=4, stride=2):
      net = slim.fully_connected(z, 1024)
      net = slim.fully_connected(net, 7 * 7 * 128)
      net = tf.reshape(net, [-1, 7, 7, 128])
      net = slim.conv2d_transpose(net, 64)
      net = slim.conv2d_transpose(net, 1,
                                  normalizer_fn=None,
                                  activation_fn=None)
      net = out_activation_fn(net)
      return net
    
def discriminator(x, cat_dim, cont_dim, fix_cont_std=True):
  """Discriminate real vs. fake and predict latents from an input image.
  
  Args:
    x: Tensor of images
    cat_dim: dimension of categorical variable
    cont_dim: number of continuous latent variables
    fix_cont_std: whether to fix the standard deviation of the approximate
      posterior for the continuous latent variables to 1.0 
      
  Returns:
    q_real: predicted distribution over real vs. fake
    q_cat: predicted distribution over categorical variable
    q_cont: predicted distribution over continuous variable
  """
  with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      activation_fn=tf.nn.leaky_relu):
    with slim.arg_scope([slim.conv2d], kernel_size=4, stride=2):
      net = slim.conv2d(x, 64, normalizer_fn=None)
      net = slim.conv2d(net, 128, normalizer_fn=slim.batch_norm)
      net = slim.flatten(net)
      net = slim.fully_connected(net, 1024, normalizer_fn=slim.batch_norm)

      # Logits for binary real vs. fake GAN discriminator
      logits_real = slim.fully_connected(net, 1, activation_fn=None)
      q_real = tfd.Bernoulli(logits=logits_real)

      # Recognition network for latent variables has an additional layer 
      encoder = slim.fully_connected(net, 128, normalizer_fn=slim.batch_norm)

      # Compute logits for each category of categorical latent
      logits_cat = slim.fully_connected(encoder, cat_dim, activation_fn=None)
      q_cat = tfd.Categorical(logits=logits_cat)

      # Compute mean and variance for Gaussian posterior of continuous latents
      cont_vars = slim.fully_connected(encoder, cont_dim * 2, activation_fn=None)
      cont_mu = cont_vars[:, :cont_dim]
      if fix_cont_std:
        cont_sigma = tf.ones_like(cont_mu)
      else:
        cont_sigma = tf.nn.softplus(cont_vars[:, cont_dim:])
      q_cont = tfd.Normal(loc=cont_mu, scale=cont_sigma)
      return q_real, q_cat, q_cont

## 파라메타 정의

In [0]:
# infogan 파라메타 
#  Incompressible noise dimensionality (not reconstructed)
noise_dim = 62
#  Dimensionality of the single categorical variable
cat_dim = 10
#  Number of continuous latent variables
cont_dim = 2
#  Weighting on InfoGAN regularization term
info_reg_coeff = 1.0

# 하이퍼파라메타
#  Generator/discriminator learning rate
gen_lr = 1e-3 # 생성기 Learning rate
disc_lr = 2e-4 # 판별기 Learning rate
NB=128 # batch size

In [5]:
#데이터셋 
tf.reset_default_graph()
images_np = mnist.train.images
images_np = images_np.reshape(-1, 28, 28, 1)
dataset = tf.data.Dataset.from_tensor_slices(images_np)
dataset = dataset.shuffle(100).repeat()
dataset = dataset.apply(
    tf.contrib.data.batch_and_drop_remainder(NB))
images = dataset.make_one_shot_iterator().get_next()

# Generate samples from prior
z_cat = tfd.Categorical(logits=tf.zeros([NB, cat_dim])).sample()
z_cat_one_hot = tf.one_hot(z_cat, cat_dim)

z_noise = tfd.Uniform(low=-tf.ones([NB, noise_dim]),
                      high=tf.ones([NB, noise_dim])).sample()
z_cont = tfd.Uniform(low=-tf.ones([NB, cont_dim]),
                     high=tf.ones([NB, cont_dim])).sample()
# Concatenate incompressible noise, discrete latents, and continuous latents
z = tf.concat([z_noise, z_cat_one_hot, z_cont], axis=1)

# Construct generator and discriminator networks
with slim.arg_scope(
    [slim.fully_connected, slim.conv2d_transpose],
    weights_initializer=tf.random_normal_initializer(stddev=0.02)):
  with slim.arg_scope(
      [slim.conv2d],
      weights_initializer=tf.truncated_normal_initializer(stddev=0.02)):
    with tf.variable_scope('gnet'):
      with slim.arg_scope([slim.batch_norm], is_training=True):
        generated = generator(z)
    with tf.variable_scope('gnet', reuse=True):
      with slim.arg_scope([slim.batch_norm], is_training=False):
        generated_test = generator(z)
    with tf.variable_scope('dnet'):
      d_real, cat_real, cont_real = discriminator(images, cat_dim, cont_dim)
    with tf.variable_scope('dnet', reuse=True):
      d_fake, cat_fake, cont_fake = discriminator(generated, cat_dim, cont_dim)

W0825 16:19:25.754715 140166383638400 deprecation.py:323] From <ipython-input-5-df9439984869>:7: batch_and_drop_remainder (from tensorflow.contrib.data.python.ops.batching) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.
W0825 16:19:25.761837 140166383638400 deprecation.py:323] From <ipython-input-5-df9439984869>:8: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
W0825 16:19:27.182322 140166383638400 deprecation.py:323] From <ipython-input-5-df9439984869>:11: Categorical.__init__ (from tensorflow.python.ops.distributions.categorical) is deprecated and will be removed 

In [6]:
# Normal GAN objectives
real_labels = tf.ones([NB, 1])
fake_labels = tf.zeros([NB, 1]) 
gan_d_loss = -(tf.reduce_mean(d_real.log_prob(real_labels)) + 
               tf.reduce_mean(d_fake.log_prob(fake_labels)))
gan_g_loss = -tf.reduce_mean(d_fake.log_prob(real_labels))

# InfoGAN loss
log_prob_cat = tf.reduce_sum(cat_fake.log_prob(z_cat)) / NB
log_prob_cont = tf.reduce_sum(cont_fake.log_prob(z_cont)) / NB
infogan_loss = -info_reg_coeff * (log_prob_cat + log_prob_cont)

# Generator and discriminator loss are the same as a GAN but with the added
# regularization to reconstruct a subset of latents from the image
d_loss = gan_d_loss + infogan_loss
g_loss = gan_g_loss + infogan_loss

# Optimization
global_step = tf.train.get_or_create_global_step()
dvars = tf.contrib.framework.get_variables('dnet')
gvars = tf.contrib.framework.get_variables('gnet')
gen_opt = tf.train.AdamOptimizer(gen_lr, 0.5)
disc_opt = tf.train.AdamOptimizer(disc_lr, 0.5)
train_gen_op = gen_opt.minimize(g_loss, global_step, var_list=gvars)
update_ops = (tf.get_collection(tf.GraphKeys.UPDATE_OPS))
with tf.control_dependencies(update_ops + [train_gen_op]):
  train_op = disc_opt.minimize(d_loss, global_step, var_list=dvars)

W0825 16:19:29.152923 140166383638400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
'''
from IPython.display import display
import matplotlib.pyplot as plt
plt.ioff()
def display_imgs(gens):
  fig, axs = plt.subplots(1, 10, figsize=(10, 1))
  for i in range(10):
    axs.flat[i].imshow(gens[i].squeeze(), interpolation='none', cmap='gray')
    axs.flat[i].axis('off');
  return fig, axs
'''

"\nfrom IPython.display import display\nimport matplotlib.pyplot as plt\nplt.ioff()\ndef display_imgs(gens):\n  fig, axs = plt.subplots(1, 10, figsize=(10, 1))\n  for i in range(10):\n    axs.flat[i].imshow(gens[i].squeeze(), interpolation='none', cmap='gray')\n    axs.flat[i].axis('off');\n  return fig, axs\n"

## Training

In [12]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
steps = 1000

outs = []
for i in range(steps+1):
  out = sess.run([train_op, global_step, log_prob_cat, log_prob_cont, infogan_loss, gan_d_loss, gan_g_loss, generated])
  outs.append(out[1:-1])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


## Visualization
### Infogan의 특징을 확인해보면 Discrete는 각 MNIST의 label의 feature를 보여주며 Continuoust는 rotation, thickness등의 feature를 보여준다.  

In [0]:
np_z = sess.run(z)
# C를 interpolation해서 보여준다.
for interp in ('continuous-1', 'continuous-2', 'discrete'):
  for i in range(4):
    z_new = np_z.copy()
    z_new[:] = z_new[i:i+1]
    if 'continuous' in interp:
      idx = int(interp.split('-')[1])
      z_new[:10, -idx] = np.linspace(-2, 2, 10)
    else:
      z_new[:cat_dim, noise_dim:noise_dim+cat_dim] = np.eye(cat_dim)
    samples = sess.run(generated_test, {z: z_new})
    display_imgs(samples);
    plt.suptitle(interp + ' interpolation')
    plt.show()

## Reference 
---
### 구현 저자 : Ben Poole, Kumar Krishna Agrawal  

[https://colab.research.google.com/drive/1JkCI_n2U2i6DFU8NKk3P6EkPo3ZTKAaq#scrollTo=qCa2lSXCohCX](https://colab.research.google.com/drive/1JkCI_n2U2i6DFU8NKk3P6EkPo3ZTKAaq#scrollTo=qCa2lSXCohCX)  
### 참고하기 좋은 코드:
[https://github.com/jiema58/InfoGan-Mnist/blob/master/InfoGan.ipynb](https://github.com/jiema58/InfoGan-Mnist/blob/master/InfoGan.ipynb)  
[https://github.com/openai/InfoGAN](https://github.com/openai/InfoGAN)  
### Infogan 이론 자료 :  
[1]InfoGAN 논문 [https://arxiv.org/abs/1606.03657](https://arxiv.org/abs/1606.03657)  
[2]infoGAN 이론 설명(영문) [https://towardsdatascience.com/infogan-generative-adversarial-networks-part-iii-380c0c6712cd](https://towardsdatascience.com/infogan-generative-adversarial-networks-part-iii-380c0c6712cd)   
[3]infoGAN 이론 설명(한글) [https://taeoh-kim.github.io/blog/generative-models-part-2-improvedganinfoganebgan](https://taeoh-kim.github.io/blog/generative-models-part-2-improvedganinfoganebgan)  
### 이미지자료 참고 :
[1] https://github.com/NRauschmayr/InfoGAN_Gluon/blob/master/InfoGAN.ipynb
 